# Data Cleaning for Projections

The historical data from the CO Secretary of State's office is not uniform -- all the spreadsheets are formatted differently from year to year, some years have precinct-by-precinct data while others do not, the fields are different, etc.

This notebook will clean all that into a standard format and then dump that data in a Pandas-friendly CSV file that can be accessed later. Doing this year by year.

In [1]:
import pandas as pd
import glob
import re
import string

reg_dir = '../data/registration/'
res_dir = '../data/results/'

## Registration Data

The voter registration data has a uniform format, so a single function should do the trick.

In [2]:
def clean_registration_data(year):
    #columns = ['DISTRICT', 'COUNTY', 
    #       'ACN-ACTIVE', 'APV-ACTIVE', 'DEM-ACTIVE', 'GRN-ACTIVE', 'LBR-ACTIVE', 'REP-ACTIVE',
    #   'UAF-ACTIVE', 'UNI-ACTIVE', 'TOTAL-ACTIVE', 
    #       'ACN-INACTIVE', 'APV-INACTIVE', 'DEM-INACTIVE', 'GRN-INACTIVE',
    #   'LBR-INACTIVE', 'REP-INACTIVE', 'UAF-INACTIVE', 'UNI-INACTIVE', 'TOTAL-INACTIVE', 
    #       'ACN-PREREG', 'APV-PREREG',
    #   'DEM-PREREG', 'GRN-PREREG', 'LBR-PREREG', 'REP-PREREG', 'UAF-PREREG', 'UNI-PREREG', 'TOTAL-PREREG',
    #       'TOTAL']
    print(year)
    df = pd.read_excel(reg_dir+'raw/{}.xlsx'.format(year), 'State Senate Districts', 
                                        header=0)#, names=columns)

    # empty squares have no voters
    df = df.fillna(0)
    df.head()

    # Remove each "Total" County instance
    #df.drop(df.loc[df['COUNTY']==0].index, inplace=True)
    
    rename_dict = {}
    for col in df.columns:
        rename_dict[col] = col.upper()
        
    df = df.rename(columns=rename_dict)
    df.head()

    df['COUNTY'] = df['COUNTY'].str.upper()

    df.to_csv(reg_dir+'cleaned/{}.csv'.format(year))

In [3]:
filepaths = glob.glob(reg_dir+'raw/*.xlsx')
years = [re.search(reg_dir+'raw/(.*).xlsx', filepath).group(1) for filepath in filepaths]
print(years)
for year in years:
    clean_registration_data(year)

['2016', '2012', '2014', '2018']
2016
2012
2014
2018


## 2018

Clean up the registration data. This involves collapsing columns of different non-D/R voters to a single "Other" category, removing intermediate rows with totals, etc.

In [ ]:
columns = ['DISTRICT', 'COUNTY', 
           'ACN-ACTIVE', 'APV-ACTIVE', 'DEM-ACTIVE', 'GRN-ACTIVE', 'LBR-ACTIVE', 'REP-ACTIVE',
       'UAF-ACTIVE', 'UNI-ACTIVE', 'TOTAL-ACTIVE', 
           'ACN-INACTIVE', 'APV-INACTIVE', 'DEM-INACTIVE', 'GRN-INACTIVE',
       'LBR-INACTIVE', 'REP-INACTIVE', 'UAF-INACTIVE', 'UNI-INACTIVE', 'TOTAL-INACTIVE', 
           'ACN-PREREG', 'APV-PREREG',
       'DEM-PREREG', 'GRN-PREREG', 'LBR-PREREG', 'REP-PREREG', 'UAF-PREREG', 'UNI-PREREG', 'TOTAL-PREREG',
           'TOTAL']

df = pd.read_excel(reg_dir+'raw/2018.xlsx', 'State Senate Districts', 
                                    header=2, names=columns)

# empty squares have no voters
df = df.fillna(0)

# Remove each "Total" County instance
df.drop(df.loc[df['COUNTY']==0].index, inplace=True)

collapsed_columns = {'ACN-ACTIVE' : 'OTHER-ACTIVE', 
                     'APV-ACTIVE' : 'OTHER-ACTIVE', 
                     'GRN-ACTIVE' : 'OTHER-ACTIVE', 
                     'LBR-ACTIVE' : 'OTHER-ACTIVE', 
                     'UAF-ACTIVE' : 'OTHER-ACTIVE', 
                     'UNI-ACTIVE' : 'OTHER-ACTIVE', 
                     'ACN-INACTIVE' : 'OTHER-INACTIVE', 
                     'APV-INACTIVE' : 'OTHER-INACTIVE', 
                     'DEM-INACTIVE' : 'OTHER-INACTIVE', 
                     'GRN-INACTIVE' : 'OTHER-INACTIVE',
                     'LBR-INACTIVE' : 'OTHER-INACTIVE', 
                     'REP-INACTIVE' : 'OTHER-INACTIVE', 
                     'UAF-INACTIVE' : 'OTHER-INACTIVE', 
                     'UNI-INACTIVE' : 'OTHER-INACTIVE',
                     'ACN-PREREG' : 'OTHER-PREREG', 
                     'APV-PREREG' : 'OTHER-PREREG',
                     'DEM-PREREG' : 'OTHER-PREREG', 
                     'GRN-PREREG' : 'OTHER-PREREG', 
                     'LBR-PREREG' : 'OTHER-PREREG', 
                     'REP-PREREG' : 'OTHER-PREREG', 
                     'UAF-PREREG' : 'OTHER-PREREG', 
                     'UNI-PREREG' : 'OTHER-PREREG',
                     'DISTRICT' : 'DISTRICT', 
                     'COUNTY' : 'COUNTY', 
                     'DEM-ACTIVE' : 'DEM-ACTIVE', 
                     'REP-ACTIVE' : 'REP-ACTIVE', 
                     'TOTAL-ACTIVE' : 'TOTAL-ACTIVE', 
                     'DEM-INACTIVE' : 'DEM-INACTIVE', 
                     'REP-INACTIVE' : 'REP-INACTIVE', 
                     'TOTAL-INACTIVE' : 'TOTAL-INACTIVE', 
                     'DEM-PREREG' : 'DEM-PREPREG', 
                     'REP-PREREG' : 'REP-PREREG', 
                     'TOTAL-PREREG' : 'TOTAL-PREREG',
                     'TOTAL' : 'TOTAL'
                    }

df = df.groupby(collapsed_columns, axis=1).sum()

df['COUNTY'] = df['COUNTY'].str.upper()

df.to_csv(reg_dir+'cleaned/2018.csv')

Clean the 2018 results, isolating the Senate district votes and reformatting the district string to match the formatting in the registration dataframe.

In [ ]:
df = pd.read_excel(res_dir+'raw/2018.xlsx')

# Convert the formatting for uniformity
df = df[df['OFFICE/BALLOT ISSUE NUMBER'].str.contains('State Senate')]
df['OFFICE/BALLOT ISSUE NUMBER'] = df['OFFICE/BALLOT ISSUE NUMBER'].str.replace('State Senate - District', 'SD')
df.rename(columns={'OFFICE/BALLOT ISSUE NUMBER' : 'OFFICE', 'CANDIDATE/JUDGE/BALLOT TITLE' : 'CANDIDATE'})

# Isolate the county data
county_vals = (df.COUNTY != 'TOTAL')
df = df[county_vals]
df = df.drop(columns=['NO VOTES', 'NO PERCENT'])

df.to_csv(res_dir+'cleaned/2018.csv')

## 2016

Clean the registration data.